- 간트차트 기준으로 올라간 영상을 기준으로 테스트 해보기 
- 추가된 컬럼: 채널명

In [31]:
# CSS Selector로 교체하기
###### https://www.youtube.com/@hare_u/shorts
# Selenium WebDriver 설정 (Chrome 사용)
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

s = Service("C:\\chromedriver\\chromedriver.exe")
driver = webdriver.Chrome(service=s)

# YouTube에 접속
driver.get("https://www.youtube.com/shorts/t0m2UUwbF08?feature=share")
time.sleep(5)  # 페이지 로드를 위한 대기 시간

# 채널 접속
go_chnl = driver.find_elements(By.CSS_SELECTOR, '#text > a')
if go_chnl:
    go_chnl[0].click()
time.sleep(4)

# 채널 이름 가져오기
chnl_name_elements = driver.find_elements(By.CSS_SELECTOR, 'div#channel-name div#text')
chnl_name = chnl_name_elements[0].text if chnl_name_elements else '직접 입력하기'
print(chnl_name)
driver.back()
WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#text')))


# 동영상 제목 가져오기
titles_elements = driver.find_elements(By.CSS_SELECTOR, '#overlay > reel-player-header-renderer > h2 > yt-formatted-string')
titles = [title.text.strip() if title.text.strip() else "직접기입" for title in titles_elements]
print(titles[0])
# "더보기" 버튼 클릭
more_button_xpath = f"/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[3]/div/ytd-reel-video-renderer[1]/div[4]/ytd-reel-player-overlay-renderer/div[2]/div/div[4]"
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, more_button_xpath))).click()
time.sleep(5)
print('worked')
# "추가설명창" 열기
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '#items > ytd-menu-service-item-renderer.style-scope.ytd-menu-popup-renderer.iron-selected'))
).click()
print('추가설명창 열림')
time.sleep(5)

# 태그, 조회수, 업로드 날짜 수집
tags_elements = driver.find_elements(By.CSS_SELECTOR, '#description > yt-formatted-string > a')
tags_text = [tag.text.replace('#', '') for tag in tags_elements]
like = driver.find_element(By.CSS_SELECTOR, '#factoids > factoid-renderer:nth-child(1) > div > span.YtwFactoidRendererValue > span').text
views_text = driver.find_element(By.CSS_SELECTOR, '#factoids > factoid-renderer:nth-child(2) > div > span.YtwFactoidRendererValue > span').text
upload_date_text = driver.find_element(By.CSS_SELECTOR, '#factoids > factoid-renderer:nth-child(3) > div').text
upload_date = upload_date_text.replace('\n', ' ')

# 데이터 저장을 위한 리스트 초기화
all_video_data = []

# 데이터 리스트에 추가
all_video_data.append(i + 1, titles, chnl_name, tags_text, like, views_text, upload_date)

# 확인을 위한 출력
for data in all_video_data:
    print(data)

driver.quit()

######################################

# 각 비디오 별로 추가 정보 수집
videos_xpath = '//*[@id="dismissible"]'
videos = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, videos_xpath)))
print('제목 개수: ',len(titles), '비디오 개수: ',len(videos))

for i, video in enumerate(videos):
    video.click()  # 비디오 클릭
    time.sleep(2) # 페이지 로드 대기

# for i, video in range(45):
#     video[i].click()  # 비디오 클릭
#     time.sleep(2) # 페이지 로드 대기    
    
    # "더보기" 버튼 클릭
    more_button_xpath = f'//*[@id="menu-button"]'
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, more_button_xpath))).click()
    time.sleep(5)
    print('worked')
    # "추가설명창" 열기
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '#items > ytd-menu-service-item-renderer.style-scope.ytd-menu-popup-renderer.iron-selected'))
    ).click()
    print('추가설명창 열림')
    time.sleep(5)
# 
#     # 태그, 조회수, 업로드 날짜 수집
    tags_elements = driver.find_elements(By.CSS_SELECTOR, '#description > yt-formatted-string > a')
    tags_text = [tag.text.replace('#', '') for tag in tags_elements]
    like = driver.find_element(By.CSS_SELECTOR, '#factoids > factoid-renderer:nth-child(1) > div > span.YtwFactoidRendererValue > span').text
    views_text = driver.find_element(By.CSS_SELECTOR, '#factoids > factoid-renderer:nth-child(2) > div > span.YtwFactoidRendererValue > span').text
    upload_date_text = driver.find_element(By.CSS_SELECTOR, '#factoids > factoid-renderer:nth-child(3) > div').text
    upload_date = upload_date_text.replace('\n', ' ')
    
# 
    # 데이터 리스트에 추가
    all_video_data.append([i + 1, titles[i], tags_text, like, views_text, upload_date])
    print('*' * 100)
    print(all_video_data[-1])
    print('*' * 100)
    # 뒤로 가기
    driver.back()
    time.sleep(3) # 페이지 로드 대기
    # print('뒤로가가기')
# 
# DataFrame 생성
df = pd.DataFrame(all_video_data, columns=['순서', '제목', '태그','좋아요', '조회수', '업로드 날짜'])

# Excel과 CSV 파일로 저장
df.to_excel('Gamtalktalk_youtube_data.xlsx', index=False)
df.to_csv('Gamtalktalk_youtube_data.csv', index=False)

print('-' * 100)
print("Data saved to '겜톡톡' 데이터 크롤링 완료")
print('-' * 100)

# 일단 스탑 

직접 입력하기
브롤스타즈 보석이💎 공짜(?) 😯!!!
worked
추가설명창 열림


TypeError: list.append() takes exactly one argument (7 given)

In [48]:
# CSS Selector로 교체하기
###### https://www.youtube.com/@hare_u/shorts
# Selenium WebDriver 설정 (Chrome 사용)
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def convert_k_to_number(input_str):
    if '천' in input_str:
        number = float(input_str.replace('천', '').strip()) * 1000
    elif '만' in input_str:
        number = float(input_str.replace('만', '').strip()) * 10000
    else:
        number = float(input_str)
    return int(number)


s = Service("C:\\chromedriver\\chromedriver.exe")
driver = webdriver.Chrome(service=s)

# YouTube에 접속
driver.get("https://www.youtube.com/shorts/t0m2UUwbF08?feature=share")
time.sleep(5)  # 페이지 로드를 위한 대기 시간

# 채널 접속
go_chnl = driver.find_elements(By.CSS_SELECTOR, '#text > a')
if go_chnl:
    go_chnl[0].click()
time.sleep(4)

# 채널 이름을 가져오기 위해 해당 요소가 나타날 때까지 대기
chnl_name_element = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.XPATH, "/html/body/ytd-app/div[1]/ytd-page-manager/ytd-browse/div[3]/ytd-c4-tabbed-header-renderer/tp-yt-app-header-layout/div/tp-yt-app-header/div/div[2]/div/div[1]/div/div[1]/ytd-channel-name/div/div/yt-formatted-string"))
)
chnl_name = chnl_name_element.text if chnl_name_element else '채널 이름 없음'
print(chnl_name)
driver.back()
time.sleep(2)



# 동영상 제목 가져오기
titles_elements = driver.find_elements(By.CSS_SELECTOR, '#overlay > reel-player-header-renderer > h2')
titles = [title.text for title in titles_elements if title.text.strip() != '']
print(titles)
# "더보기" 버튼 클릭
more_button_xpath = f"/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[3]/div/ytd-reel-video-renderer[1]/div[4]/ytd-reel-player-overlay-renderer/div[2]/div/div[4]"
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, more_button_xpath))).click()
time.sleep(5)
print('worked')
# "추가설명창" 열기
WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CSS_SELECTOR, '#items > ytd-menu-service-item-renderer.style-scope.ytd-menu-popup-renderer.iron-selected'))
).click()
print('추가설명창 열림')
time.sleep(5)

# 태그, 조회수, 업로드 날짜 수집
tags_elements = driver.find_elements(By.CSS_SELECTOR, '#description > yt-formatted-string > a')
tags_text = [tag.text.replace('#', '') for tag in tags_elements]
like = driver.find_element(By.CSS_SELECTOR, '#factoids > factoid-renderer:nth-child(1) > div > span.YtwFactoidRendererValue > span').text
like = convert_k_to_number(like)
views_text = driver.find_element(By.CSS_SELECTOR, '#factoids > factoid-renderer:nth-child(2) > div > span.YtwFactoidRendererValue > span').text
upload_date_text = driver.find_element(By.CSS_SELECTOR, '#factoids > factoid-renderer:nth-child(3) > div').text
upload_date = upload_date_text.replace('\n', ' ')

# 데이터 저장을 위한 리스트 초기화
all_video_data = []

# 데이터 리스트에 유효한 제목만 추가
for i, title in enumerate(titles):
    all_video_data.append((i + 1, title, chnl_name, tags_text, like, views_text, upload_date))


# 확인을 위한 출력
for data in all_video_data:
    print(data)

틱톡★꽐루
['브롤스타즈 보석이💎 공짜(?) 😯!!!']
worked
추가설명창 열림
(1, '브롤스타즈 보석이💎 공짜(?) 😯!!!', '틱톡★꽐루', [], 4100, '191,108', '2월 23일 2024년')


In [ ]:
# 가져올 채널 제목 
# https://www.youtube.com/shorts/t0m2UUwbF08?feature=share
# https://www.youtube.com/shorts/oaud8n0ZoFM
# https://www.youtube.com/shorts/NCmR2ZWaVsQ
# https://www.youtube.com/shorts/V33wIascWBE

In [ ]:
    urls = [
        "https://www.youtube.com/shorts/t0m2UUwbF08?feature=share",
        "https://www.youtube.com/shorts/oaud8n0ZoFM",
        "https://www.youtube.com/shorts/NCmR2ZWaVsQ",
        "https://www.youtube.com/shorts/V33wIascWBE",
        'https://youtube.com/shorts/wFBJNM3A27w?si=Kv76bDpQUjkep7yp',
        'https://youtube.com/shorts/pmQUVXUNxbo?si=sWp4bhSDDnPmj35y',
        'https://youtube.com/shorts/lNwlhHHl31w?si=j-LuIJApVM7Z5y7n',
        'https://youtube.com/shorts/09K4826jcGU?feature=shared',
        'https://www.youtube.com/@ndlover7',
        'https://www.youtube.com/@TODEM',
        'https://www.youtube.com/@holit92'
    ]

뚜뱀이 : 5만별 지급 완료
https://youtube.com/shorts/wFBJNM3A27w?si=Kv76bDpQUjkep7yp

행님 : 13만원 지급 완료
https://youtube.com/shorts/pmQUVXUNxbo?si=sWp4bhSDDnPmj35y

틱톡꽐루 : 35만원 지급 완료
https://youtube.com/shorts/lNwlhHHl31w?si=j-LuIJApVM7Z5y7n

하래유 : 100만원
https://youtube.com/shorts/09K4826jcGU?feature=shared

ND러너 : 60만원 4월 25일 진행
https://www.youtube.com/@ndlover7

토뎀이 : 70만원 4월 26일 진행
https://www.youtube.com/@TODEM

홀릿 : 550만원 5월 4일
https://www.youtube.com/@holit92

In [7]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def convert_k_to_number(input_str):
    if '천' in input_str:
        number = float(input_str.replace('천', '').strip()) * 1000
    elif '만' in input_str:
        number = float(input_str.replace('만', '').strip()) * 10000
    else:
        number = float(input_str)
    return int(number)

s = Service("C:\\chromedriver\\chromedriver.exe")
driver = webdriver.Chrome(service=s)

# 데이터 저장을 위한 글로벌 리스트
all_video_data = []

# 스크래핑할 URL 리스트
urls = [
        "https://www.youtube.com/shorts/t0m2UUwbF08?feature=share",
        "https://www.youtube.com/shorts/oaud8n0ZoFM",
        "https://www.youtube.com/shorts/NCmR2ZWaVsQ",
        "https://www.youtube.com/shorts/V33wIascWBE",
        'https://youtube.com/shorts/wFBJNM3A27w?si=Kv76bDpQUjkep7yp',
        'https://youtube.com/shorts/pmQUVXUNxbo?si=sWp4bhSDDnPmj35y',
        'https://youtube.com/shorts/lNwlhHHl31w?si=j-LuIJApVM7Z5y7n',
        'https://youtube.com/shorts/09K4826jcGU?feature=shared',
        # 업로드 아직 안됨
        # 'https://www.youtube.com/@ndlover7',
        # 'https://www.youtube.com/@TODEM',
        # 'https://www.youtube.com/@holit92'
]

for url in urls:
    driver.get(url)
    time.sleep(5)  # 페이지 로드를 위한 대기 시간
    
    # 채널 접속
    go_chnl = driver.find_elements(By.CSS_SELECTOR, '#text > a')
    if go_chnl:
        go_chnl[0].click()
    time.sleep(4)
    
    # 채널 이름 가져오기
    chnl_name_element = WebDriverWait(driver, 10).until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, '#page-header > yt-page-header-renderer > yt-page-header-view-model > div > div.page-header-view-model-wiz__page-header-headline > div > yt-dynamic-text-view-model > h1'))
    )
    chnl_name = chnl_name_element.text if chnl_name_element else '채널 이름 없음'
    # print(chnl_name)
    driver.back()
    time.sleep(2)

    # 동영상 제목 가져오기
    titles_elements = driver.find_elements(By.CSS_SELECTOR, '#overlay > reel-player-header-renderer > h2')
    titles = [title.text for title in titles_elements if title.text.strip() != '']
    # print(titles)
    
    # "더보기" 버튼 클릭
    more_button_xpath = f"/html/body/ytd-app/div[1]/ytd-page-manager/ytd-shorts/div[3]/div/ytd-reel-video-renderer[1]/div[4]/ytd-reel-player-overlay-renderer/div[2]/div/div[4]"
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, more_button_xpath))).click()
    time.sleep(5)
    print('worked')
    # "추가설명창" 열기
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '#items > ytd-menu-service-item-renderer.style-scope.ytd-menu-popup-renderer.iron-selected'))
    ).click()
    print('추가설명창 열림')
    time.sleep(5)
    
    # 태그, 조회수, 업로드 날짜 수집
    tags_elements = driver.find_elements(By.CSS_SELECTOR, '#description > yt-formatted-string > a')
    tags_text = [tag.text.replace('#', '') for tag in tags_elements]
    like = driver.find_element(By.CSS_SELECTOR, '#factoids > factoid-renderer:nth-child(1) > div > span.YtwFactoidRendererValue > span').text
    like = convert_k_to_number(like)
    views_text = driver.find_element(By.CSS_SELECTOR, '#factoids > factoid-renderer:nth-child(2) > div > span.YtwFactoidRendererValue > span').text
    upload_date_text = driver.find_element(By.CSS_SELECTOR, '#factoids > factoid-renderer:nth-child(3) > div').text
    upload_date = upload_date_text.replace('\n', ' ')
    print('*'* 50)
    print(titles, chnl_name, tags_text, like, views_text, upload_date)
    print('*'* 50)
    
    # 데이터 추가
    for i, title in enumerate(titles):
        all_video_data.append((len(all_video_data) + 1, title, chnl_name, tags_text, like, views_text, upload_date))

# 확인을 위한 출력
for data in all_video_data:
    print(data)

driver.quit()

# 데이터를 Excel 파일로 저장
df = pd.DataFrame(all_video_data, columns=['순서', '제목', '채널명', '태그', '좋아요', '조회수', '업로드 날짜'])
df.to_excel('INF_youtube_data.xlsx', index=False)
print('Saved')

worked


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.119)
Stacktrace:
	GetHandleVerifier [0x00007FF7304E7072+63090]
	(No symbol) [0x00007FF730452CC2]
	(No symbol) [0x00007FF7302EEC65]
	(No symbol) [0x00007FF7302CCA7C]
	(No symbol) [0x00007FF73035D687]
	(No symbol) [0x00007FF730372AC1]
	(No symbol) [0x00007FF730356D83]
	(No symbol) [0x00007FF7303283A8]
	(No symbol) [0x00007FF730329441]
	GetHandleVerifier [0x00007FF7308E25CD+4238285]
	GetHandleVerifier [0x00007FF73091F72D+4488493]
	GetHandleVerifier [0x00007FF730917A0F+4456463]
	GetHandleVerifier [0x00007FF7305C05B6+953270]
	(No symbol) [0x00007FF73045E58F]
	(No symbol) [0x00007FF730459264]
	(No symbol) [0x00007FF73045939B]
	(No symbol) [0x00007FF730449BD4]
	BaseThreadInitThunk [0x00007FFE4161257D+29]
	RtlUserThreadStart [0x00007FFE417EAA48+40]


In [17]:
import pandas as pd
file_path = 'INF_youtube_data.xlsx'
data = pd.read_excel(file_path)

In [19]:
print(data)

   순서                                        제목    채널명  \
0   1                     브롤스타즈 보석이💎 공짜(?) 😯!!!  틱톡★꽐루   
1   2               브롤스타즈 공짜 하이퍼차지 가즈아!!!!!!!!!    너굴몬   
2   3                    🤔아직도 로블 기본아바타 쓰는사람 없지?  멜로우TV   
3   4  '무료'로 브롤에 현질하는 방법 공개! ㅋㅋㅋ #브롤스타즈 #마인크래프트    아이고   
4   5                    보석을 공짜로 얻는 방법?! #브롤스타즈    뚜뱀이   
5   6                               보석 쉽게 얻는 방법     행님   
6   7                     (10명) 깊카 추첨으로 뿌립니다🤩!!  틱톡★꽐루   
7   8                                현질을 하고 싶어요    하래유   

                           태그    좋아요      조회수        업로드 날짜  
0                          []   4100  191,166  2월 23일 2024년  
1  ['브롤스타즈', '브롤', '브롤스타즈게임']   1900   45,566  2월 29일 2024년  
2                          []  14000  352,124  3월 22일 2024년  
3                  ['shorts']   7200  200,388  3월 30일 2024년  
4                          []    110    5,058  3월 30일 2024년  
5                          []   1100   45,157   4월 6일 2024년  
6                          []   4000  101,2